In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import cmocean as cm
import xarray as xr
import numpy as np
import IPython.display

import cosima_cookbook as cc
import pandas as pd
pd.set_option("display.max_rows", 50)

In [6]:
session = cc.database.create_session()

In [3]:
expList = cc.querying.get_experiments(session)

In [13]:
expList.loc[expList["experiment"].str.contains("qian")]

,experiment,ncfiles
37,01deg_jra55v13_ryf9091_qian_wtlp,400
134,01deg_jra55v13_ryf9091_qian_ctrl,1440
182,01deg_jra55v13_ryf9091_qian_wthp,2400
183,01deg_jra55v13_ryf9091_qian_wthmp,2000


In [5]:
#experiment = "01deg_jra55v13_ryf9091_qian_wthmp"
#experiment = "01deg_jra55v13_ryf9091_qian_ctrl"
experiment = "01deg_jra55v13_ryf9091_qian_wthp"

In [7]:
fileList = cc.querying.get_ncfiles(session, experiment)

In [23]:
varList = cc.querying.get_variables(session, experiment=experiment, frequency='1 monthly')

In [51]:
fileList

,ncfile,index_time
0,output1000/ice/OUTPUT/iceh.2151-01.nc,2023-03-14 03:28:09.140048
1,output1000/ice/OUTPUT/iceh.2151-02.nc,2023-03-14 03:17:33.284809
2,output1000/ice/OUTPUT/iceh.2151-03.nc,2023-03-14 03:18:23.492479
3,output1000/ocean/o2i.nc,2023-03-14 03:17:54.640911
4,output1000/ocean/ocean.nc,2023-03-14 03:22:32.084957
...,...,...
1995,output999/ocean/ocean_grid.nc,2023-03-14 03:37:20.678783
1996,output999/ocean/ocean_heat_budget.nc,2023-03-14 03:15:46.310639
1997,output999/ocean/ocean_month.nc,2023-03-14 03:44:36.007570
1998,output999/ocean/ocean_scalar.nc,2023-03-14 03:36:05.699356


In [20]:
i = 0

In [25]:
varList.iloc[i:i+10]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
20,blkmask,ice grid block mask,None,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
21,bmf_u,Bottom u-stress via bottom drag,N/m^2,1 monthly,output1035/ocean/ocean_month.nc,time: mean,200,2110-01-01 00:00:00,2160-01-01 00:00:00
22,bmf_v,Bottom v-stress via bottom drag,N/m^2,1 monthly,output1035/ocean/ocean_month.nc,time: mean,200,2110-01-01 00:00:00,2160-01-01 00:00:00
23,buoyfreq2_wt,Squared buoyancy frequency at T-cell bottom,1/s^2,1 monthly,output1035/ocean/ocean.nc,time: mean,200,2110-01-01 00:00:00,2160-01-01 00:00:00
24,congel_m,congelation ice growth,cm/day,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,time: mean,600,2110-01-01 00:00:00,2160-01-01 00:00:00
25,divu_m,strain rate (divergence),%/day,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,time: mean,600,2110-01-01 00:00:00,2160-01-01 00:00:00
26,dxt,T cell width through middle,m,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
27,dxu,U cell width through middle,m,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
28,dyt,T cell height through middle,m,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
29,dyu,U cell height through middle,m,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00


In [26]:
i+=10

In [31]:
varList.loc[varList["name"].str.lower().str.contains("pressure")]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end


In [7]:
variable = 'salt'

salt = cc.querying.getvar(experiment, variable, session)#, start_time="1990-01-01", end_time="2019-01-01")

variable = 'temp'

temp = cc.querying.getvar(experiment, variable, session)

u = cc.querying.getvar(experiment, "u", session)

v = cc.querying.getvar(experiment, "v", session)

sea_level = cc.querying.getvar(experiment, "sea_level", session)

pbot_t = cc.querying.getvar(experiment, "pbot_t", session)

In [8]:
ds = xr.Dataset(coords = {"time": temp.time,
                          "xt_ocean": temp.xt_ocean.values,
                          "yt_ocean": [-60.0, -29.98],
                          "yu_ocean": [-60.02, -30.02],
                          "xu_ocean": u.xu_ocean.values,
                         "st_ocean": temp.st_ocean.values})

In [9]:
ds["u"] = u.sel(yu_ocean = [-60, -30], method="nearest").chunk({"time":720, "st_ocean":75, "yu_ocean":2, "xu_ocean":3600})
ds["v"] = v.sel(yu_ocean = [-60, -30], method="nearest").chunk({"time":720, "st_ocean":75, "yu_ocean":2, "xu_ocean":3600})
ds["temp"] = temp.sel(yt_ocean = [-60, -30], method="nearest").chunk({"time":720, "st_ocean":75, "yt_ocean":2, "xt_ocean":3600})
ds["salt"] = salt.sel(yt_ocean = [-60, -30], method="nearest").chunk({"time":720, "st_ocean":75, "yt_ocean":2, "xt_ocean":3600})
ds["sea_level"] = sea_level.sel(yt_ocean = [-60, -30], method="nearest").chunk({"time":720, "yt_ocean":2, "xt_ocean":3600})
ds["pbot_t"] = pbot_t.sel(yt_ocean = [-60, -30], method="nearest").chunk({"time":720, "yt_ocean":2, "xt_ocean":3600})

In [ ]:
ds.to_netcdf("/g/data/g40/an1995/CAM_PG_project/SLICE_30S_60S_01deg_jra55v13_ryf9091_qian_wthp.nc")

In [ ]:
1

# Qian's Meltwater Experiment

In [11]:
experiment = "01deg_jra55v13_ryf9091_qian_wthmp"

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [15]:
variable = 'v'

darray = cc.querying.getvar(experiment, variable, session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

VariableNotFoundError: No files were found containing 'v' in the '01deg_jra55v13_ryf9091_qian_wthmp' experiment

In [13]:
varList = cc.querying.get_variables(session, experiment=experiment, frequency='1 monthly')

In [17]:
varList.loc[varList["name"].str.lower().str.contains("u")]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
8,ULAT,U grid center latitude,degrees_north,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
9,ULON,U grid center longitude,degrees_east,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
21,bmf_u,Bottom u-stress via bottom drag,N/m^2,1 monthly,output1035/ocean/ocean_month.nc,time: mean,200,2110-01-01 00:00:00,2160-01-01 00:00:00
23,buoyfreq2_wt,Squared buoyancy frequency at T-cell bottom,1/s^2,1 monthly,output1035/ocean/ocean.nc,time: mean,200,2110-01-01 00:00:00,2160-01-01 00:00:00
25,divu_m,strain rate (divergence),%/day,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,time: mean,600,2110-01-01 00:00:00,2160-01-01 00:00:00
27,dxu,U cell width through middle,m,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
29,dyu,U cell height through middle,m,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,None,600,2110-01-01 00:00:00,2160-01-01 00:00:00
45,fswup_m,upward solar flux,W/m^2,1 monthly,output1035/ice/OUTPUT/iceh.2159-12.nc,time: mean,600,2110-01-01 00:00:00,2160-01-01 00:00:00
47,grid_xu_ocean,ucell longitude,degrees_E,1 monthly,output1035/ocean/ocean.nc,None,200,2110-01-01 00:00:00,2160-01-01 00:00:00
49,grid_yu_ocean,ucell latitude,degrees_N,1 monthly,output1035/ocean/ocean.nc,None,200,2110-01-01 00:00:00,2160-01-01 00:00:00


In [5]:
variable = 'v'

darray = cc.querying.getvar(experiment, variable, session, start_time="1990-01-01", end_time="2019-01-01")

0.3.0


<xarray.DataArray 'v' (time: 349, st_ocean: 75, yu_ocean: 2700, xu_ocean: 3600)>
dask.array<concatenate, shape=(349, 75, 2700, 3600), dtype=float32, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 ... 89.92 89.96 90.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 1989-12-16T12:00:00 ... 2018-12-16T12:00:00
Attributes:
    long_name:      j-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_water_y_velocity
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

In [6]:
v_zonalSlices = darray.sel(yu_ocean=[-30, -60], method="nearest")

In [ ]:
v_zonalSlices.to_netcdf("~/01deg_jra55v13_ryf9091_qian_wthmp/v_zonalSlices.nc")

In [ ]:
variable = 'u'

darray = cc.querying.getvar(experiment, variable, session, start_time="1990-01-01", end_time="2019-01-01")
darray

In [ ]:
u_zonalSlices = darray.sel(yu_ocean=[-30, -60], method="nearest")

In [ ]:
u_zonalSlices.to_netcdf("~/SLICE_01deg_jra55v140_iaf_cycle4/u_zonalSlices.nc")